In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
### import data
ordert = pd.read_csv('order_products__train.csv')
orderp = pd.read_csv('order_products__prior.csv')

orders = pd.read_csv('orders.csv')

products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

In [3]:
### product imformation
m1 = pd.merge(products, aisles, how='left', on=['aisle_id'])
products_new = pd.merge(m1, departments, how='left', on=['department_id'])
print("Size of product :", products_new.shape[0])
products_new.head()

Size of product : 49688


,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


In [4]:
### orders and order_products__prior
orders_products = pd.merge(orderp, orders, how='left', on=['order_id'])
orders_products = orders_products.fillna(0)
orders_products.head(10)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0000
1,2,28985,2,1,202279,prior,3,5,9,8.0000
2,2,9327,3,0,202279,prior,3,5,9,8.0000
3,2,45918,4,1,202279,prior,3,5,9,8.0000
4,2,30035,5,0,202279,prior,3,5,9,8.0000
5,2,17794,6,1,202279,prior,3,5,9,8.0000
6,2,40141,7,1,202279,prior,3,5,9,8.0000
7,2,1819,8,1,202279,prior,3,5,9,8.0000
8,2,43668,9,0,202279,prior,3,5,9,8.0000
9,3,33754,1,1,205970,prior,16,5,17,12.0000


In [5]:
train_orders_products = pd.merge(ordert, orders, how='left', on=['order_id'])
train_orders_products.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0000
1,1,11109,2,1,112108,train,4,4,10,9.0000
2,1,10246,3,0,112108,train,4,4,10,9.0000
3,1,49683,4,0,112108,train,4,4,10,9.0000
4,1,43633,5,1,112108,train,4,4,10,9.0000


In [6]:
f = {'up_mean_days_since_prior_order': np.mean,
     'up_var_days_since_prior_order': np.var,
     'up_count_days_since_prior_order': "count"} 
g1 = orders_products.groupby(["user_id", "product_id"])["days_since_prior_order"].aggregate(f).reset_index()
### numpy variance的公式的分母是(n-1)，因此NaN應該補0
g1 = g1.fillna(0)
print(g1.isnull().sum())
g1.head()

user_id                            0
product_id                         0
up_mean_days_since_prior_order     0
up_var_days_since_prior_order      0
up_count_days_since_prior_order    0
dtype: int64


,user_id,product_id,up_mean_days_since_prior_order,up_var_days_since_prior_order,up_count_days_since_prior_order
0,1,196,17.6000,116.7111,10
1,1,10258,19.5556,88.2778,9
2,1,10326,28.0000,0.0000,1
3,1,12427,17.6000,116.7111,10
4,1,13032,21.6667,58.3333,3


In [8]:
### 加入train and test顧客消費的資訊
orders_new = orders[(orders.eval_set != "prior")]
data_1 = pd.merge(g1, orders_new, how='left', on=["user_id"])

ordert_new = train_orders_products[["user_id", "product_id", "reordered"]]
data_2 = pd.merge(data_1, train_orders_products[["user_id", "product_id", "reordered"]], how='left', on=["user_id", "product_id"])
data_2.head()

,user_id,product_id,up_mean_days_since_prior_order,up_var_days_since_prior_order,up_count_days_since_prior_order,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,17.6000,116.7111,10,1187899,train,11,4,8,14.0000,1.0000
1,1,10258,19.5556,88.2778,9,1187899,train,11,4,8,14.0000,1.0000
2,1,10326,28.0000,0.0000,1,1187899,train,11,4,8,14.0000,nan
3,1,12427,17.6000,116.7111,10,1187899,train,11,4,8,14.0000,nan
4,1,13032,21.6667,58.3333,3,1187899,train,11,4,8,14.0000,1.0000


In [9]:
print(data_2.isnull().sum())
data_2.reordered = data_2.reordered.replace(to_replace = "nan", value = 0)
data_2.head()

user_id                                   0
product_id                                0
up_mean_days_since_prior_order            0
up_var_days_since_prior_order             0
up_count_days_since_prior_order           0
order_id                                  0
eval_set                                  0
order_number                              0
order_dow                                 0
order_hour_of_day                         0
days_since_prior_order                    0
reordered                          12479129
dtype: int64


,user_id,product_id,up_mean_days_since_prior_order,up_var_days_since_prior_order,up_count_days_since_prior_order,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,reordered
0,1,196,17.6000,116.7111,10,1187899,train,11,4,8,14.0000,1.0000
1,1,10258,19.5556,88.2778,9,1187899,train,11,4,8,14.0000,1.0000
2,1,10326,28.0000,0.0000,1,1187899,train,11,4,8,14.0000,0.0000
3,1,12427,17.6000,116.7111,10,1187899,train,11,4,8,14.0000,0.0000
4,1,13032,21.6667,58.3333,3,1187899,train,11,4,8,14.0000,1.0000
